In [2]:
!pwd

/workspaces/cosc470/notebooks


In [1]:
import torch
from torchvision import datasets, transforms

mnist_train = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

mnist_test = datasets.MNIST(
    root='/workspaces/cosc470/data',
    train=False,
    download=True,
    transform=transforms.ToTensor()
)



In [6]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=3, bias=False)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(50 * 5 * 5, 100, bias=False)
        self.fc2 = nn.Linear(100, 10, bias=False)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 50 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

model = SimpleCNN()
bad_prediction = model.forward(torch.randn(1, 1, 28, 28))
print(bad_prediction)
real_image = mnist_train[0][0].unsqueeze(0)
print(mnist_train[0][1])
good_prediction = model.forward(real_image)
print(good_prediction)

tensor([[0.1012, 0.0954, 0.0971, 0.0849, 0.0881, 0.1062, 0.1040, 0.1167, 0.1046,
         0.1017]], grad_fn=<SoftmaxBackward0>)
5
tensor([[0.1014, 0.0964, 0.1014, 0.0982, 0.0993, 0.0997, 0.1020, 0.1023, 0.1005,
         0.0990]], grad_fn=<SoftmaxBackward0>)


In [7]:
from torch.utils.data import DataLoader

# Set hyperparameters
learning_rate = 0.01
batch_size = 10
epochs = 20

# DataLoader for training
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")
    correct = 0
    for testimage in mnist_test:
        a = testimage[0].unsqueeze(0)
        a = model.forward(a)
        if torch.argmax(a) == testimage[1]:
            correct += 1
    print(f"Test Accuracy: {100 * correct / len(mnist_test):.2f}%")

Epoch [1/20], Loss: 1.9991
Test Accuracy: 82.54%
Epoch [2/20], Loss: 1.6367
Test Accuracy: 89.06%
Epoch [3/20], Loss: 1.5416
Test Accuracy: 94.92%
Epoch [4/20], Loss: 1.5116
Test Accuracy: 96.40%
Epoch [5/20], Loss: 1.4991
Test Accuracy: 97.25%
Epoch [6/20], Loss: 1.4922
Test Accuracy: 97.60%
Epoch [7/20], Loss: 1.4882
Test Accuracy: 97.61%
Epoch [8/20], Loss: 1.4854
Test Accuracy: 98.04%
Epoch [9/20], Loss: 1.4830
Test Accuracy: 97.86%
Epoch [10/20], Loss: 1.4812
Test Accuracy: 98.39%
Epoch [11/20], Loss: 1.4796
Test Accuracy: 98.31%
Epoch [12/20], Loss: 1.4779
Test Accuracy: 98.23%
Epoch [13/20], Loss: 1.4770
Test Accuracy: 98.65%
Epoch [14/20], Loss: 1.4764
Test Accuracy: 98.49%
Epoch [15/20], Loss: 1.4751
Test Accuracy: 98.72%
Epoch [16/20], Loss: 1.4744
Test Accuracy: 98.59%
Epoch [17/20], Loss: 1.4737
Test Accuracy: 98.52%
Epoch [18/20], Loss: 1.4733
Test Accuracy: 98.67%
Epoch [19/20], Loss: 1.4725
Test Accuracy: 98.59%
Epoch [20/20], Loss: 1.4719
Test Accuracy: 98.76%
